In [ ]:
#| default_exp core

# OUScope

> The telescope control is provided through the `Telescope` class which provides state tracking and low level methods - forming a basic API layer. The higher level functions are implemented as separate functions construced with the Telescope class API.

In [ ]:
#| hide
from nbdev.showdoc import *
from nbdev import *
from fastcore.basics import patch

In [ ]:
#| exporti
import logging
import requests
from requests import session
import configparser
from os.path import expanduser
import diskcache
from bs4 import BeautifulSoup
import json
import time

In [ ]:
#| exporti
def cleanup(s):
    return s.encode('ascii','ignore').decode('ascii','ignore')

In [ ]:
#| export
class Telescope:
    '''
    Main API class
    '''
    
    url='https://www.telescope.org/'
    cameratypes={
        'constellation':'1',
        'galaxy':       '2',
        'cluster':      '3',
        'planet':'5',
        'coast':'6',
        'pirate':'7',
    }

    REQUESTSTATUS_TEXTS={
        1: "New",
        2: "New, allocated",
        3: "Waiting",
        4: "In progress",
        5: "Reallocate",
        6: "Waiting again",
        7: "Complete on site",
        8: "Complete",
        9: "Hold",
        10: "Frozen",
        20: "Expired",
        21: "Expired w/CJobs",
        22: "Cancelled",
        23: "Cancelled w/CJobs",
        24: "Invalid",
        25: "Never rises",
        26: "Other error",
    }
    
    def __init__(self, user, passwd, cache='.cache/jobs'):
        self.s=None
        self.user=user
        self.passwd=passwd
        self.tout=60
        self.retry=15
        self.login()
        self.cache=cache


In [ ]:
#| exporti
@patch
def login(self: Telescope):
    log = logging.getLogger(__name__)
    payload = {'action': 'login',
               'username': self.user,
               'password': self.passwd,
               'stayloggedin': 'true'}
    log.debug('Get session ...')
    self.s=session()
    log.debug('Logging in ...')
    self.s.post(self.url+'login.php', data=payload)

In [ ]:
#| exporti
@patch
def logout(self: Telescope):
    if self.s is None :
        self.s.post(self.url+'logout.php')
        self.s=None

In [ ]:
#| login
config = configparser.ConfigParser()
config.read(expanduser('~/.config/telescope.ini'))

scope=Telescope(config['telescope.org']['user'],
                    config['telescope.org']['password'],
                    config['cache']['jobs'])
astrometryAPIkey=config['astrometry.net']['apikey']

wcscache=diskcache.Cache(config['cache']['wcs'])
seqcache=diskcache.Cache(config['cache']['seq'])

scope.__dict__

{'s': <requests.sessions.Session>,
 'user': 'jochym',
 'passwd': 'uJH5iLqUEuc49ju',
 'tout': 60,
 'retry': 15,
 'cache': '.cache/jobs'}

In [ ]:
#| export
@patch
def get_user_requests(self: Telescope, sort='rid', folder=1):
    '''
    Get all user requests from folder (Inbox=1 by default),
    sorted by sort column ('rid' by default). 
    Possible sort columns are: 'rid', 'object', 'completion'
    The data is returned as a list of dictionaries.
    '''

    #fetch first batch        
    params={
        'limit': 100,
        'sort': sort,
        'folderid': folder}

    rq = self.s.post(self.url+"api-user.php", {'module': "request-manager", 
                                               'request': "1-get-list-own",
                                               'params' : json.dumps(params)})
    res=[]
    dat=json.loads(rq.content)
    total=int(dat['data']['totalRequests'])
    res+=dat['data']['requests']

    # Fetch the rest
    params['limit']=total-len(res)
    params['startAfterRow']=len(res)
    rq = self.s.post(self.url+"api-user.php", {'module': "request-manager", 
                                               'request': "1-get-list-own",
                                               'params' : json.dumps(params)})

    dat=json.loads(rq.content)
    total=int(dat['data']['totalRequests'])
    res+=dat['data']['requests']
    return res

In [ ]:
#| login
rqs = scope.get_user_requests()
print(f'User {scope.user} has {len(rqs)} requests. Recent few:')
for rq in rqs[:10]:
    print(f'{rq["id"]}: {rq["objectname"]}')

User jochym has 1439 requests. Recent few:
744748: DX Vul
744747: BI Her
744746: SS Cyg
744745: V1223 Sgr
744385: LX Cyg
744384: EQ Lyr
744383: DQ Vul
744382: BI Her
744381: V686 Cyg
744380: IP Cyg


In [ ]:
#| export
@patch
def get_user_folders(self: Telescope):
    '''
    Get all user folders. Returns list of dictionaries.
    '''
    rq = self.s.post(self.url+"api-user.php", {'module': "request-manager", 
                                               'request': "0-get-my-folders"})
    return json.loads(rq.content)['data']

In [ ]:
#| login
scope.get_user_folders()

[{'id': '1', 'creationtime': '0', 'name': 'Inbox', 'count': '1439'},
 {'id': '2', 'creationtime': '0', 'name': 'Favourites', 'count': None},
 {'id': '3', 'creationtime': '0', 'name': 'Archive', 'count': '447'},
 {'id': '4', 'creationtime': '0', 'name': 'Trash', 'count': '52'},
 {'id': '461',
  'creationtime': '1407254495',
  'name': 'Complete',
  'count': '13'}]

In [ ]:
#| export
@patch
def get_obs_list(self: Telescope, t=None, dt=1, filtertype='', camera='', hour=16, minute=0):
    '''Get the dt days of observations taken no later then time in t.

        Input
        ------
        t  - end time in seconds from the epoch
            (as returned by time.time())
        dt - number of days, default to 1
        filtertype - filter by type of filter used
        camera - filter by the camera/telescope used

        Output
        ------
        Returns a list of JobIDs (int) for the observations.

    '''

    assert(self.s is not None)

    if t is None :
        t=time.time()-time.timezone


    st=time.gmtime(t-86400*dt)
    et=time.gmtime(t)

    d=st.tm_mday
    m=st.tm_mon
    y=st.tm_year
    de=et.tm_mday
    me=et.tm_mon
    ye=et.tm_year

    log = logging.getLogger(__name__)
    log.debug('%d/%d/%d -> %d/%d/%d', d,m,y,de,me,ye)

    try :
        telescope=self.cameratypes[camera.lower()]
    except KeyError:
        telescope=''

    searchdat = {
        'sort1':'completetime',
        'sort1order':'desc',
        'searchearliestcom[]':[d, m, y, str(hour),str(minute)],
        'searchlatestcom[]':  [de,me,ye,str(hour),str(minute)],
        'searchstatus[]':['1'],
        'resultsperpage':'1000',
        'searchfilter':filtertype,
        'searchtelescope':telescope,
        'submit':'Go'
    }

    headers = {'Content-Type': 'application/x-www-form-urlencoded'}


    request = self.s.post(self.url+'v3job-search-query.php',
                     data=searchdat, headers=headers)
    soup = BeautifulSoup(request.text,'lxml')

    jlst=[]
    for l in soup.findAll('tr'):
        try :
            a=l.find('a').get('href')
        except AttributeError :
            continue
        jid=a.rfind('jid')
        if jid>0 :
            jid=a[jid+4:].split('&')[0]
            jlst.append(int(jid))
    return jlst

In [ ]:
#| login 
scope.get_obs_list()

[399267, 399206, 399512, 399485, 399484, 399500]